Import Necessary Libraries

In [546]:
import statsapi as api
import pandas as pd
import tempfile
import os
import re
import requests
from bs4 import BeautifulSoup, Comment


Input Player Name and Year

In [547]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
year = input(f'Enter Year: ')
player_name = f'{first_name.lower()} {last_name.lower()}'


fetch player_id 

In [548]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

661388
158


Gather Player hand

In [549]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
print(batter_hand)

R


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand

In [550]:
date = '2024-04-20'
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

sPitcher_info = api.lookup_player(sPitcher_name)
sPitcher_id = sPitcher_info[0]['id']
sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_id)
print(home)

True
745190
138
571945
R
2889
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified 

In [551]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])

    if x.get('label') == 'First pitch':
        game_time = x['value']

    if x.get('label') == 'Umpires':
        ump_info = x['value']
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        print(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            proj_ump_ids = []
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_ump_ids.append(ump_id)
            print(proj_ump_ids)

            

print(temp)
print(wind_speed)
print(game_time)


427413
58
11
1:16 PM.


fetch player game data: 
- batting position

In [552]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])
        print (batting_position)


2


fetch game player stats
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [553]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'k: {k}')
print(f'bb: {bb}')

    

ab: 6
h: 3
r: 2
rbi: 1
hr: 0
k: 1
bb: 0


Format player URL for finding streak averages

In [554]:
first_initial = last_name[0].lower()
search_url = f'https://www.mlb.com/players#{first_initial}'

response = requests.get(search_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

div = soup.find('div', id='players-index')

if div:
    player_links = div.find_next_sibling('div')

print(player_links)





None
